In [0]:
import random
import collections

In [307]:
SUIT_ID = {0:'s', 1:'c', 2:'h', 3:'d'}
VALUE_ID = {i - 2 : str(i) for i in range(2,10)}
VALUE_ID.update({8:'T', 9:'J', 10:'Q', 11:'K', 12:'A'})
ID_SUIT = {val: key for key, val in SUIT_ID.items()}
ID_VALUE = {val: key for key, val in VALUE_ID.items()}


class Card(object):

  def _init_sv(self, suit, value):
    self.suit_id = ID_SUIT[suit]
    self.value_id = ID_VALUE[value]
    self.id = self.suit_id*13 + self.value_id

  def __init__(self, *args, **kwargs):
    if kwargs.get('id') is not None:
      self.id = kwargs.get('id')
      if self.id < 0 or self.id > 52:
        raise ValueError('Card initializerd with id %d', id)
      self.suit_id = self.id // 13
      self.value_id = self.id - self.suit_id*13
    elif kwargs.get('suit') is not None:
      self._init_sv(kwargs.get('suit'), kwargs.get('value'))
    elif args[0] is not None:
      value = args[0][0]
      suit = args[0][1]
      self._init_sv(suit, value)
    else:
      raise ValueError("Card initialized without id, suit+value, or text representation.")
    self.human_str = VALUE_ID[self.value_id]+SUIT_ID[self.suit_id]

  def __repr__(self):
    return self.human_str
    
  def __eq__(self, other):
    return self.id == other.id

  def __lt__(self, other):
    if self.value_id == other.value_id:
      return self.suit_id < other.suit_id
    else:
      return self.value_id < other.value_id

  
print(Card(id=5))
print(Card(suit='h',value='T'))
print(Card('Qh'))
print(Card('Qc') < Card('Kc'), Card('Qc') < Card('Js'), Card('Qc') < Card('Qc'))

7s
Th
Qh
True False False


In [0]:
for i in range(52):
  print(Card(id=i), Card(id=i).id)

for suit in ['s', 'c', 'h', 'd']:
  for value in [str(i) for i in range(2,10)] + ['T', 'J', 'Q', 'K', 'A']:
      print(Card(suit=suit, value=value), Card(suit=suit, value=value).id)


In [334]:
class Pile(object):
  def __init__(self, cards=None):
    self.cards = []
    if cards is not None:
      if type(cards) == list:
        self.cards = cards
      elif type(cards) == str:
        self.cards = [Card(cards[i:i+2]) for i in range(0,len(cards),2)]

  def Remove(self, card):
    self.cards.remove(card)
  
  def Move(self, card, pile):
    self.Remove(card)
    pile.cards.append(card)

  def Draw(self, count, pile):
    if count < 0 or count > len(self.cards):
      raise ValueError("Invalid dealing count, %d but Deck has %d cards", count, len(self.cards))
    for i in range(count):
        pile.cards.append(self.cards.pop(-1))

  def Shuffle(self):
    random.shuffle(self.cards)

  def __contains__(self, cards):
    return all([card in self.cards for card in cards])

  def __iter__(self): 
      return list.__iter__(self.cards)
  
  def __repr__(self):
    return str(self.cards)

  def __getitem__(self, i):
    return self.cards[i]

  def __add__(self, other):
    return Pile(self.cards + other.cards)
  
  def __len__(self):
    return len(self.cards)

class Deck(Pile):
  def __init__(self):
    super().__init__()
    for i in range(52):
      self.cards.append(Card(id=i))

print(Pile('QhTd'))
print(Deck()[5])

[Qh, Td]
7s


In [330]:
D = Deck()
D.Shuffle()
D.cards
P = Pile()
D.Draw(4, P)
print([Card(id=c.id) for c in P] in P)
print([Card(id=c.id) for c in P] in D)

True
False


In [320]:
class Range(object):
  def __init__(self, *args, **kwargs):
    self.hands = []
    for pile_str in args:
      self.hands.append(Pile(cards=pile_str))

  def Add(self, hand):
    if hand not in self.hands:
      self.hands.append(hand)

  def __iter__(self): 
      return list.__iter__(self.hands)

  def __repr__(self):
    return str(self.hands)
  
  def __len__(self):
    return len(self.hands)

  def __getitem__(self, i):
    return self.hands[i]

  def RangeUnion(self, pile):
    ret = Range()
    for hand in self.hands:
      if hand in pile:
        ret.Add(hand)
    return ret

  def RandomHandFromRange(self):
    return
    

print(Range('QhTh', 'QdTd', 'QsTs'))
print(Range('QhTh', 'QdTd', 'QsTs', 'QcTc'))
p = Pile(cards='QhThQsTsAd6d')
r = Range('QhTh', 'QdTd', 'QsTs', 'QcTc')
r.RangeUnion(p)

[[Qh, Th], [Qd, Td], [Qs, Ts]]
[[Qh, Th], [Qd, Td], [Qs, Ts], [Qc, Tc]]


[[Qh, Th], [Qs, Ts]]

In [0]:
# In order of best to worst
HAND_CATEGORIES = ['straight_flush',
                    'quads',
                    'full_house',
                    'flush',
                    'straight',
                    'trips',
                    'two_pair',
                    'pair',
                    'high_card']

class HoldemEvaluator(object):
  class HoldemHand(object):
    def __init__(self, category, value):
      if type(value) == int:
        self.comp_value = value
      else:
        comp_value = 0
        for val in value:
          comp_value = comp_value*14
          comp_value += val
        self.comp_value = comp_value
      self.value = value
      self.category = category
      self.category_value = list(reversed(HAND_CATEGORIES)).index(self.category)
        
    def __lt__(self, other):
      if self.category_value < other.category_value:
        return True
      elif self.category_value == other.category_value:
        return self.comp_value < other.comp_value
      else:
        return False

    def __eq__(self, other):
      return self.category_value == other.category_value and self.category_value == other.category_value

    def __repr__(self):
      return self.Formatted()
    
    def Formatted(self):
      if self.category == 'straight_flush':
        format_str = 'Straight Flush, %s high' 
      elif self.category == 'quads':
        format_str = 'Four of a Kind %ss, %s kicker' 
      elif self.category == 'full_house':
        format_str = 'Full House %ss full of %ss' 
      elif self.category == 'flush':
        format_str = 'Flush, %s high' 
      elif self.category == 'straight':
        format_str = 'Straight, %s high' 
      elif self.category == 'trips':
        format_str = 'Three of a kind %ss, %s%s kicker'  
      elif self.category == 'two_pair':
        format_str = 'Two Pair %ss and %ss, %s kicker' 
      elif self.category == 'pair':
        format_str = 'Pair %s, %s%s%s kicker' 
      elif self.category == 'high_card':
        format_str = '%s%s%s%s%s high'
      if type(self.value) == int:
        return format_str % VALUE_ID[self.value]
      return format_str % tuple([VALUE_ID[val] for val in self.value])


  def __init__(self, pile):
    self.pile = Pile(cards=sorted(pile, reverse=True))

  def Process(self):
    # Straight Flush
    self.biggest_straight_flush = None
    curr_straight_flush_values = collections.defaultdict(lambda : -1)
    num_in_flush_sequence = collections.defaultdict(lambda : 1)
    # Flush, Straight Flush
    self.biggest_suits = {}

    # Flush
    self.suit_counts = collections.defaultdict(lambda : 0)
    self.biggest_flush = None

    # Straight
    self.biggest_straight = None # value id of biggest straight
    curr_straight_value = -1
    num_in_sequence = 1

    # Pair, 2-Pair, Trips, Full-house, Quads, Kickers
    self.value_counts = collections.defaultdict(lambda : 0)
    self.biggest_pair = None
    self.second_biggest_pair = None
    self.biggest_trips = None
    self.second_biggest_trips = None
    self.biggest_quads = None
  
    for card in self.pile:
      self.suit_counts[card.suit_id] += 1
      self.value_counts[card.value_id] += 1

      # Flush Counts
      if card.suit_id not in self.biggest_suits:
        self.biggest_suits[card.suit_id] = card.value_id
      
      # Straight Sequences
      if self.biggest_straight is None:
        if card.value_id == curr_straight_value - 1:
          num_in_sequence += 1
          if num_in_sequence == 5:
            self.biggest_straight = card.value_id + 4
          # The Wheel
          elif num_in_sequence == 4 and card.value_id == 0 and self.pile[0].value_id == 12:
            self.biggest_straight = 3
        else:
          num_in_sequence = 1

        curr_straight_value = card.value_id

      # Straight Flush Sequences
      if self.biggest_straight_flush is None:
        curr_straight_flush_value = curr_straight_flush_values[card.suit_id]
        if card.value_id == curr_straight_flush_value - 1:
          num_in_flush_sequence[card.suit_id] += 1
          if num_in_flush_sequence[card.suit_id] == 5:
            self.biggest_straight_flush = card.value_id + 4
          # The Wheel Flush
          elif num_in_flush_sequence[card.suit_id] == 4 and card.value_id == 0 and self.biggest_suits[card.suit_id] == 12:
            self.biggest_straight_flush = 3
        else:
          curr_straight_flush_values[card.suit_id] = card.value_id
        curr_straight_flush_values[card.suit_id] = card.value_id

    # Pair processing
    for value_id in sorted(self.value_counts.keys(), reverse=True):
      count = self.value_counts[value_id]
      if count == 2:
        if self.biggest_pair is None:
          self.biggest_pair = value_id
        elif self.second_biggest_pair is None:
          self.second_biggest_pair = value_id
      elif count == 3:
        if self.biggest_trips is None:
          self.biggest_trips = value_id
        elif self.second_biggest_trips is None:
          self.second_biggest_trips = value_id
      elif count == 4:
        if self.biggest_quads is None:
          self.biggest_quads = value_id
    # Flush processing:
    for suit_id, count in self.suit_counts.items():
      if count >= 5:
        flush_value = self.biggest_suits[suit_id]
        if self.biggest_flush is None:
          self.biggest_flush = flush_value
        elif self.biggest_flush < flush_value:
          self.biggest_flush = flush_value

    
  def BiggestStraightFlush(self):
    return self.biggest_straight_flush
  def BiggestQuads(self):
    if self.biggest_quads is not None:
      kickers = [i for i in sorted(self.value_counts.keys(), reverse=True) if i != self.biggest_quads]
      return (self.biggest_quads, kickers[0])
  def BiggestFullHouse(self):
    if self.biggest_trips is not None:
      pairs = []
      if self.second_biggest_trips is not None:
        pairs.append(self.second_biggest_trips)
      if self.biggest_pair is not None:
        pairs.append(self.biggest_pair)
      if pairs:
        return [self.biggest_trips, max(pairs)]
  def BiggestFlush(self):
    return self.biggest_flush
  def BiggestStraight(self):
    return self.biggest_straight
  def BiggestTrips(self):
    if self.biggest_trips:
      kickers = [i for i in sorted(self.value_counts.keys(), reverse=True) if i != self.biggest_trips]
      if len(kickers) >= 2:
        return (self.biggest_trips, kickers[0], kickers[1])
  def BiggestTwoPair(self):
    if self.second_biggest_pair is not None:
      kickers = [i for i in sorted(self.value_counts.keys(), reverse=True) 
                 if i != self.biggest_pair and i != self.second_biggest_pair]
      if len(kickers) >= 1:
        return (self.biggest_pair, self.second_biggest_pair, kickers[0])
  def BiggestPair(self):
    if self.biggest_pair:
      kickers = [i for i in sorted(self.value_counts.keys(), reverse=True) 
                 if i != self.biggest_pair]
      if len(kickers) >=3:
        return (self.biggest_pair, kickers[0], kickers[1], kickers[2])
  def BiggestHighCard(self):
    kickers = [i for i in sorted(self.value_counts.keys(), reverse=True)]
    if len(kickers) >= 5:
      return kickers[:5]

  def BestHand(self):
    self.Process()
    tup = self.BiggestStraightFlush()
    if tup is not None:
      return self.HoldemHand('straight_flush', tup)
    tup = self.BiggestQuads()
    if tup is not None:
      return self.HoldemHand('quads', tup)
    tup = self.BiggestFullHouse()
    if tup is not None:
      return self.HoldemHand('full_house', tup)
    tup = self.BiggestFlush()
    if tup is not None:
      return self.HoldemHand('flush', tup)
    tup = self.BiggestStraight()
    if tup is not None:
      return self.HoldemHand('straight', tup)
    tup = self.BiggestTrips()
    if tup is not None:
      return self.HoldemHand('trips', tup)
    tup = self.BiggestTwoPair()
    if tup is not None:
      return self.HoldemHand('two_pair', tup)
    tup = self.BiggestPair()
    if tup is not None:
      return self.HoldemHand('pair', tup)
    tup = self.BiggestHighCard()
    if tup is not None:
      return self.HoldemHand('high_card', tup)
    raise ValueError("No best hand determined.")

In [0]:
# #Straights
h = HoldemEvaluator(Pile('KhQdTcAs2c'))
h.Process()
assert(h.BiggestStraight() is None)

h = HoldemEvaluator(Pile('3c2d4s5cAs'))
h.Process()
assert(h.BiggestStraight() == Card('5c').value_id)

h = HoldemEvaluator(Pile('Qc8s8hJdTs9h8c'))
h.Process()
assert(h.BiggestStraight() == Card('Qc').value_id)

#Straight Flushes
h = HoldemEvaluator(Pile('Qc8s8hJdTs9h8c'))
h.Process()
assert(h.BiggestStraightFlush() is None)

h = HoldemEvaluator(Pile('3c2c4c5cAc'))
h.Process()
assert(h.BiggestStraightFlush() == Card('5c').value_id)

h = HoldemEvaluator(Pile('3c2c4c5cAc6c'))
h.Process()
assert(h.BiggestStraightFlush() == Card('6c').value_id)

h = HoldemEvaluator(Pile('3c2c4c5cAcAsKsQsJsTs'))
h.Process()
assert(h.BiggestStraightFlush() == Card('As').value_id)

h = HoldemEvaluator(Pile('3c2c4c5cAc'))
h.Process()
assert(h.BiggestStraightFlush() == Card('5c').value_id)

h = HoldemEvaluator(Pile('Qc8s8hJcTc9c8c'))
h.Process()
assert(h.BiggestStraightFlush() == Card('Qc').value_id)

# Flushes
h = HoldemEvaluator(Pile('Jc8s8hQcTc9c8c'))
h.Process()
assert(h.BiggestFlush()== Card('Qc').value_id)

# Full Houses
h = HoldemEvaluator(Pile('Jc8s8hQcTcTs9c8c'))
h.Process()
fh_tup = h.BiggestFullHouse()
assert(fh_tup[0] == Card('8c').value_id and fh_tup[1] == Card('Tc').value_id)

h = HoldemEvaluator(Pile('TdTcTsThJdJcAdAcAs'))
h.Process()
fh_tup = h.BiggestFullHouse()
assert(fh_tup[0] == Card('Ac').value_id and fh_tup[1] == Card('Jc').value_id)

h = HoldemEvaluator(Pile('TdTcTsJdJcAdAc'))
h.Process()
fh_tup = h.BiggestFullHouse()
assert(fh_tup[0] == Card('Tc').value_id and fh_tup[1] == Card('Ac').value_id)

# Two Pair
h = HoldemEvaluator(Pile('TdTcTsThJdJcAdAcAs'))
h.Process()
tp_tup = h.BiggestTwoPair()
assert(tp_tup is None)

h = HoldemEvaluator(Pile('TsJdJcAdAc'))
h.Process()
tp_tup = h.BiggestTwoPair()
assert(tp_tup[0] == Card('Ac').value_id and tp_tup[1] == Card('Jc').value_id and tp_tup[2] == Card('Ts').value_id)

In [314]:
print(HoldemEvaluator(Pile('KsQsJsTs9s')).BestHand())
print(HoldemEvaluator(Pile('3c2c4c5cAc')).BestHand())

print(HoldemEvaluator(Pile('AcAdAs9h9d9c9sJhKd')).BestHand())

print(HoldemEvaluator(Pile('KsKcKdAsAc')).BestHand())
print(HoldemEvaluator(Pile('KsKcKdAsAcAdTc')).BestHand())
print(HoldemEvaluator(Pile('KsKcKdAsAcAd')).BestHand())

print(HoldemEvaluator(Pile('Ah6h5h4h3h')).BestHand())

print(HoldemEvaluator(Pile('3c2c4c5cAs')).BestHand())

print(HoldemEvaluator(Pile('KsKcKdAsQcJd')).BestHand())

print(HoldemEvaluator(Pile('AhAsKhKsJhJs')).BestHand())

print(HoldemEvaluator(Pile('AhAsKhKsJhJs')).BestHand())

print(HoldemEvaluator(Pile('AhAsKsJsTs9c')).BestHand())

print(HoldemEvaluator(Pile('6h5c4d2s7c')).BestHand())

print(HoldemEvaluator(Pile('AhAsKsJsTs9c')).BestHand() == HoldemEvaluator(Pile('AhAdKsJs9cTs')).BestHand())

Straight Flush, K high
Straight Flush, 5 high
Four of a Kind 9s, A kicker
Full House Ks full of As
Full House As full of Ks
Full House As full of Ks
Flush, A high
Straight, 5 high
Three of a kind Ks, AQ kicker
Two Pair As and Ks, J kicker
Two Pair As and Ks, J kicker
Pair A, KJT kicker
76542 high
True


In [0]:
class SimulationError(Exception):
  pass

class MonteCarloHoldemEquityCalculator(object):
  def __init__(self, sample_size=10000):
    self.sample_size = sample_size
    self.deck = Deck()
    self.players = []
    self.board = None
    self.fixed_board = Pile()
    self.num_retries_before_NaN = 50

  def AddPlayer(self, range):
    self.players.append(range)

  def _DealBoard(self):
    self.board = Pile()
    for card in self.fixed_board:
      self.deck.Move(card, self.board)
    self.deck.Draw(5 - len(self.board), self.board)    

  def SetBoard(self, pile):
    self.fixed_board = pile
  
  def _DealAllPlayers(self):
    for _ in range(self.num_retries_before_NaN):
      player_hands = []
      for idx, player_range in enumerate(self.players):
        remaining_range = player_range.RangeUnion(self.deck)
        if len(remaining_range) == 0:
          if idx == 0:
            raise SimulationError("Could not deal players within ranges in %d tries." % self.num_retries_before_NaN)
          break
        simulation_hand = random.choice(remaining_range)
        for card in simulation_hand:
          self.deck.Remove(card)
        player_hands.append(simulation_hand)
      if player_hands:
        return player_hands

    raise SimulationError("Could not deal players within ranges in %d tries." % self.num_retries_before_NaN)
        
  def ResetDecks(self):
    self.deck = Deck()
    self.deck.Shuffle()
    self.board = Pile()

  def _SimulateOneHand(self):

    for _ in range(self.num_retries_before_NaN):
      self.ResetDecks()
      self._DealBoard()
      try:
        player_hands = self._DealAllPlayers()
        break
      except SimulationError:
        pass

    if not player_hands:
      raise SimulationError("Could not generate hand outcome within ranges in %d tries." % self.num_retries_before_NaN)
    
    player_hands = [hand + self.board for hand in player_hands]
    player_made_hands = [HoldemEvaluator(hand).BestHand() for hand in player_hands]
    winner = player_made_hands.index(max(player_made_hands))
    return winner

  def ComputePlayerEquityMap(self):
    player_wins_map = {i:0 for i in range(len(self.players))}
    for _ in range(self.sample_size):
      winner = self._SimulateOneHand()
      player_wins_map[winner] += 1
    return {i: v/self.sample_size for i, v in player_wins_map.items()}



In [416]:
calculator = MonteCarloHoldemEquityCalculator(sample_size=100000)
calculator.AddPlayer(Range('AhAd'))
calculator.AddPlayer(Range('KhKd'))
calculator.SetBoard(Pile('KcQsJd'))

calculator.ComputePlayerEquityMap()


{0: 0.28845, 1: 0.71155}